<a href="https://colab.research.google.com/github/tsjendoll/KNN-Book-Recommendation/blob/main/Book_Recommendation_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-02-17 16:54:30--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  33.7MB/s    in 0.7s    

2024-02-17 16:54:32 (33.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author','year','publisher','cover_image_small', 'cover_image_medium','cover_image'],
    usecols=['isbn', 'title', 'author','year','publisher','cover_image_small', 'cover_image_medium','cover_image'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str','year' : 'str','publisher' : 'str','cover_image' : 'str'})


df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
df_books = df_books.drop(['year','publisher', 'cover_image_small', 'cover_image_medium'], axis=1)

In [ ]:
df = df_ratings.merge(df_books, on='isbn', how='left')

In [ ]:
df.head()

,user,isbn,rating,title,author,cover_image
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5.0,Rites of Passage,Judith Rae,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3.0,Help!: Level 1,Philip Prowse,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,http://images.amazon.com/images/P/0521795028.0...


In [ ]:
df.shape

(1149780, 6)

In [ ]:
# prompt: Using dataframe df: keep rows only if user appears at least 200 times
users = df['user'].value_counts()
isbn = df['isbn'].value_counts()

users = users[users >= 200].index
isbn = isbn[isbn >= 100].index

In [ ]:
df = df.loc[(df['user'].isin(users.values)) & (df['isbn'].isin(isbn.values))]

In [ ]:
df.shape

(49781, 6)

In [ ]:
df = df.drop_duplicates(['title','user'])

In [ ]:
df.shape

(49374, 6)

In [ ]:
df.isna().sum()

user             0
isbn             0
rating           0
title          238
author         238
cover_image    238
dtype: int64

In [ ]:
df = df.dropna(how='any')

In [ ]:
df.shape

(49136, 6)

In [ ]:
# prompt: Using dataframe df: create a pivot table with the index as title, columns is user, and values is rating

df_pivot = df.pivot(values='rating', index='title', columns='user').fillna(0)
df_pivot.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_matrix = csr_matrix(df_pivot.values)

In [ ]:
nn = NearestNeighbors(metric='cosine')
nn.fit(df_matrix)

NearestNeighbors(metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = [book,[]]
  distance, book_info = nn.kneighbors([df_pivot.loc[book]], 6, return_distance=True)

  recom_book_info = df_pivot.iloc[np.flip(book_info[0])[:-1]].index.to_list()
  recom_dist = list(np.flip(distance[0])[:-1])

  for r in zip(recom_book_info, recom_dist):
    recommended_books[1].append(list(r))


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
